In [2]:
import numpy as np
import numpy.fft as fft
import matplotlib.pyplot as plt
from scipy import signal
from IPython import display
from PIL import Image
import math

%matplotlib inline

In [3]:
np.random.seed(805764)

# psf
y = np.expand_dims(np.sort(np.random.randint(1, 51, 3)), axis=1)

# each image
x1 = np.expand_dims(np.sort(np.random.randint(1, 51, 3)), axis=1)
x2 = np.expand_dims(np.sort(np.random.randint(1, 51, 3)), axis=1)
x3 = np.expand_dims(np.sort(np.random.randint(1, 51, 3)), axis=1)

print(np.concatenate([y, x1, x2, x3]))

[[ 4]
 [ 9]
 [22]
 [12]
 [13]
 [30]
 [24]
 [33]
 [40]
 [14]
 [20]
 [42]]


In [38]:
# caculated convolutions
b1 = np.empty((len(y), len(x1)))
b2 = np.empty((len(y), len(x2)))
b3 = np.empty((len(y), len(x3)))

for i in range(len(y)):
    for j in range(len(x1)):
        b1[i,j] = y[i] + x1[j]
        b2[i,j] = y[i] + x2[j]
        b3[i,j] = y[i] + x3[j]
        
# each b is list of points of where it's 1: [3, 8, 10]
b1 = np.expand_dims(np.sort(b1.flatten()), axis=1)
b2 = np.expand_dims(np.sort(b2.flatten()), axis=1)
b3 = np.expand_dims(np.sort(b3.flatten()), axis=1)

print(np.concatenate([b1, b2, b3]))

[[16.]
 [17.]
 [21.]
 [22.]
 [34.]
 [34.]
 [35.]
 [39.]
 [52.]
 [28.]
 [33.]
 [37.]
 [42.]
 [44.]
 [46.]
 [49.]
 [55.]
 [62.]
 [18.]
 [23.]
 [24.]
 [29.]
 [36.]
 [42.]
 [46.]
 [51.]
 [64.]]


In [40]:
# each row of A represents an equation like y[i] + x1[j].
# each column of A represents a measurement x1, x2, x3
A = np.array([
     [1 ,0 ,0 ,1 ,0 ,0],
     [1 ,0 ,0 ,0 ,1 ,0],
     [1 ,0 ,0 ,0 ,0 ,1],
     [0 ,1 ,0 ,1 ,0 ,0],
     [0 ,1 ,0 ,0 ,1 ,0],
     [0 ,1 ,0 ,0 ,0 ,1],
     [0 ,0 ,1 ,1 ,0 ,0],
     [0 ,0 ,1 ,0 ,1 ,0],
     [0 ,0 ,1 ,0 ,0 ,1],
     [0 ,0 ,0 ,1 ,0 ,0]
])

vec = np.random.permutation(9)
rness = 3

In [41]:
# b1 shape
# b1 = A.dot()

print(x1.shape)
print(y.shape)
print(b1.shape)
print(A.shape)

b1 = A.dot(np.vstack((x1, y)))
b1 = b1[:len(b1)-1]
b1 = np.sort(b1)
bs = np.vstack((b1, np.array([0])))
besterror = sum(pow(np.vstack((b1[vec], np.array([0]))) - A.dot(np.linalg.pinv(A)).dot(np.vstack((b1[vec], np.array([0])))),2))
print('initial: ' + str(besterror))
for iteration in range(20):
    print('iteration ' + str(iteration))
    order = np.random.permutation(b1.size)
    for i in range(b1.size):
        vectemp = vec
        swapdistance = np.round(np.random.randn(1)[0] * rness)
        swapind = int(min(max(order[i] + swapdistance, 0), b1.size - 1))
        vec[order[i]] = vectemp[swapind]
        vec[swapind] = vectemp[order[i]]
        # error = sum(pow(np.vstack((b1[vec], np.array([0]))) - A.dot(np.linalg.pinv(A)).dot(np.vstack((b1[vec], np.array([0])))), 2))
        error = np.linalg.norm(np.vstack((b1[vec], np.array([0]))) - A.dot(np.linalg.pinv(A)).dot(np.vstack((b1[vec], np.array([0])))))
        if error < besterror:
            besterror = error
            print('improved: ' + str(besterror))
        else:
            vec = vectemp
    if besterror < 1e-1:
        break
            


(3, 1)
(3, 1)
(9, 1)
(10, 6)
initial: [370.66666667]
iteration 0
improved: 19.252705437591533
improved: 13.216151734399338
improved: 11.850925889754116
iteration 1
improved: 11.680943645290158
improved: 11.518101695447333
iteration 2
improved: 11.362707032706215
improved: 9.36897954837013
improved: 8.856886837057612
iteration 3
improved: 8.666666666666664
iteration 4
improved: 5.3126990493633696e-14
